In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re

# --- CONFIGURATION ---
BASE_URL = "https://econpapers.repec.org"
SEARCH_URL = "https://econpapers.repec.org/scripts/jelsearch.pf"
# Liste des codes JEL principaux (A à Z) comme mentionné dans le PDF [cite: 113]
JEL_CODES = ['E', 'F', 'G', 'H', 'I', 'J'] # Exemple réduit pour le test. Ajoutez les autres (A-Z).
START_YEAR = 2015
END_YEAR = 2025
MAX_PAPERS_PER_JEL = 200  # Limite mentionnée dans le PDF 

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'From': 'researcher@university.edu' # Bonne pratique : indiquez un contact
}

def get_paper_details(paper_url, primary_jel):
    """
    Extrait les détails d'une page individuelle d'article selon la section 2d du PDF.
    """
    try:
        response = requests.get(paper_url, headers=headers)
        if response.status_code != 200:
            return None
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 1. Titre [cite: 103]
        title_tag = soup.find('h1', class_='colored') # Structure standard EconPapers
        title = title_tag.get_text(strip=True) if title_tag else "N/A"

        # 2. Auteurs & Affiliations [cite: 104, 105]
        # Sur EconPapers, les auteurs sont souvent listés juste après le titre ou dans des balises spécifiques.
        # Cette partie nécessite souvent d'analyser le texte brut si la structure n'est pas stricte.
        authors = []
        affiliations = []
        
        # Recherche des infos auteurs (structure variable selon le type de papier)
        author_divs = soup.find_all('p', class_='small') 
        author_text = ""
        for p in author_divs:
            if "Author" in p.get_text() or "@" in p.get_text():
                author_text += p.get_text(strip=True) + " "
        
        # Extraction simplifiée (à raffiner selon le HTML exact rencontré)
        authors = author_text if author_text else "Voir lien"

        # 3. Année & Journal/Series [cite: 106, 108]
        # On cherche souvent dans le bloc de citation ou métadonnées
        text_content = soup.get_text()
        year = "N/A"
        
        # Tentative d'extraction de l'année via Regex (cherche 2015-2025)
        date_match = re.search(r'(201[5-9]|202[0-5])', text_content)
        if date_match:
            year = int(date_match.group(0))
        
        # Vérification du filtre date [cite: 106]
        if year != "N/A" and (year < START_YEAR or year > END_YEAR):
            return "OUT_OF_DATE"

        # 4. Codes JEL complets [cite: 102]
        jel_section = soup.find(string=re.compile("JEL-codes:"))
        jel_codes = "N/A"
        if jel_section:
            # Récupère le texte parent ou suivant
            jel_codes = jel_section.find_parent().get_text(strip=True).replace("JEL-codes:", "")

        # 5. Lien de téléchargement [cite: 109]
        download_section = soup.find('div', id='download')
        download_link = "N/A"
        if download_section:
            link_tag = download_section.find('a', href=True)
            if link_tag:
                download_link = BASE_URL + link_tag['href']

        return {
            "JEL Primary": primary_jel,   # [cite: 107]
            "Title": title,               # [cite: 103]
            "Authors": authors,           # [cite: 104]
            "Affiliations": "Voir Full Text", # Souvent complexe à isoler sans NLP
            "Year": year,                 # [cite: 106]
            "JEL Codes": jel_codes,       # [cite: 102]
            "Journal/Series": "Working Paper/Journal", # À parser plus finement
            "URL": paper_url              # [cite: 109]
        }

    except Exception as e:
        print(f"Erreur lors du parsing de {paper_url}: {e}")
        return None

def scrape_repec():
    all_data = []

    print(f"Début du scraping pour la période {START_YEAR}-{END_YEAR}...")

    for jel in JEL_CODES:
        print(f"--- Traitement du code JEL: {jel} ---")
        
        # Paramètres pour la recherche par code JEL
        # Note: EconPapers utilise souvent des formulaires, ici on simule une requête GET simple si possible
        # ou on itère sur les pages de catégories.
        # URL type: https://econpapers.repec.org/scripts/jelsearch.pf?text=E
        params = {'text': jel} 
        
        try:
            response = requests.get(SEARCH_URL, params=params, headers=headers)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Trouver les liens vers les papiers dans les résultats de recherche
            # Les résultats sont souvent dans des balises <li class="inlist"> ou similaires
            # Ceci est une approximation de la structure de EconPapers
            links = soup.find_all('a', href=True)
            
            paper_links = []
            for link in links:
                href = link['href']
                # Filtrer pour ne garder que les liens vers des articles (souvent /paper/ ou /article/)
                if '/paper/' in href or '/article/' in href or '/repec/' in href:
                    full_link = BASE_URL + href if href.startswith('/') else href
                    if full_link not in paper_links:
                        paper_links.append(full_link)
            
            print(f"Trouvé {len(paper_links)} papiers potentiels pour {jel}. Traitement des {MAX_PAPERS_PER_JEL} premiers...")

            count = 0
            for link in paper_links:
                if count >= MAX_PAPERS_PER_JEL:
                    break
                
                # Pause pour être poli envers le serveur (Anti-ban)
                time.sleep(random.uniform(1, 2))
                
                paper_data = get_paper_details(link, jel)
                
                if paper_data == "OUT_OF_DATE":
                    continue # On ignore ce papier mais on continue
                
                if paper_data:
                    all_data.append(paper_data)
                    count += 1
                    print(f"Scrapé ({count}): {paper_data['Title'][:30]}...")

        except Exception as e:
            print(f"Erreur globale sur le code {jel}: {e}")

    # Création du DataFrame final
    df = pd.DataFrame(all_data)
    return df

# --- EXÉCUTION ---
if __name__ == "__main__":
    df_results = scrape_repec()
    
    # Aperçu des données
    print(df_results.head())
    
    # Sauvegarde CSV comme suggéré pour l'analyse future
    df_results.to_csv("repec_data_raw.csv", index=False)
    print("Données sauvegardées dans repec_data_raw.csv")

ModuleNotFoundError: No module named 'pandas'

In [3]:
movies = response.findAll('div', class_='lister-item mode-advanced')
print(len(movies))

AttributeError: 'Response' object has no attribute 'findAll'